In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

link location https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks

In [2]:
%matplotlib inline

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [5]:
spotify_df = pd.read_csv('tracks.csv')

In [6]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [7]:
data_by_genres = pd.read_csv('data_by_genres_o.csv')
data_by_genres.head()

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,5
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,7
3,1,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,7
4,1,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,7


In [8]:
data_by_genres.dtypes

mode                  int64
genres               object
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
liveness            float64
loudness            float64
speechiness         float64
tempo               float64
valence             float64
popularity          float64
key                   int64
dtype: object

In [9]:
data_by_genres['genres'].values[0][0]

'2'

In [10]:
data_by_artist = pd.read_csv('data_by_artist_o.csv')
data_by_artist.head()

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7
3,[],"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0,1,27
4,[],"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5,1,7


In [11]:
data_by_artist.dtypes

genres               object
artists              object
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
liveness            float64
loudness            float64
speechiness         float64
tempo               float64
valence             float64
popularity          float64
key                   int64
mode                  int64
count                 int64
dtype: object

In [12]:
data_by_artist['genres'].values[0][0]

'['

In [13]:
data_0 = pd.read_csv('data_o.csv')
data_0.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [14]:
data_0.dtypes

valence             float64
year                  int64
acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
release_date         object
speechiness         float64
tempo               float64
dtype: object

In [15]:
data_by_artist['genres_upd'] = data_by_artist['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])
data_by_artist['genres_upd'].values[0][0]

'show_tunes'

In [16]:
#spotify_df['artists'] = spotify_df['artists'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])
#spotify_df['artists'].values[0][0]

In [17]:
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [18]:
spotify_df['artists'].values[0]

"['Uli']"

In [19]:
spotify_df['artists_upd_v1'].values[0][0]

'Uli'

In [20]:
spotify_df[spotify_df['artists_upd_v1'].apply(lambda x: not x)].head(5)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
164,1xEEYhWxT4WhDQdxfPCT8D,Snake Rag,20,194533,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.708,0.361,0,-11.764,0,0.0441,0.994,0.883,0.103,0.902,105.695,4,[]
170,3rauXVLOOM5BlxWqUcDpkg,Chimes Blues,14,170827,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.546,0.189,0,-15.984,1,0.0581,0.996,0.908,0.339,0.554,80.318,4,[]
172,1UdqHVRFYMZKU2Q7xkLtYc,Pickin' On Your Baby,11,197493,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.520,0.153,0,-14.042,1,0.0440,0.995,0.131,0.353,0.319,102.937,4,[]
174,0Vl2DO5U6FjgBpzCtBN3OA,Everybody Loves My Baby,10,152507,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.514,0.193,2,-13.920,0,0.2380,0.996,0.199,0.248,0.665,180.674,4,[]
180,5SvyP1ZeJX1jA7AOZD08NA,Tears,10,187227,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.359,0.357,3,-11.810,1,0.0511,0.994,0.819,0.290,0.753,205.053,4,[]


In [21]:
spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists'])

In [22]:
spotify_df['artists_upd'].dtypes

dtype('O')

In [23]:
spotify_df['artists_upd'].head()

0                ['Uli']
1    ['Fernando Pessoa']
2    ['Ignacio Corsini']
3    ['Ignacio Corsini']
4        ['Dick Haymes']
Name: artists_upd, dtype: object

In [24]:
spotify_df['name'].head()

0                                  Carve
1    Capítulo 2.16 - Banquero Anarquista
2     Vivo para Quererte - Remasterizado
3          El Prisionero - Remasterizado
4                    Lady of the Evening
Name: name, dtype: object

In [25]:
spotify_df['artists_song'] = spotify_df.apply(lambda row: str(row['artists_upd'][2:-2])+str(row['name']),axis = 1)

In [26]:
print(str(spotify_df['artists_upd'][0]))

['Uli']


In [27]:
spotify_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [28]:
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,0,-5.407,1,0.0315,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],['Miley Cyrus'],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],['Harry Styles'],Harry StylesAdore You
92524,1M4qEo4HE3PRaCOM7EXNJq,Adore You,74,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-06,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],['Harry Styles'],Harry StylesAdore You


In [29]:
spotify_df.drop_duplicates('artists_song',inplace = True)

In [30]:
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,0,-5.407,1,0.0315,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],['Miley Cyrus'],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,8,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],['Harry Styles'],Harry StylesAdore You


In [31]:
artists_exploded = spotify_df[['artists_upd','id']].explode('artists_upd')

In [32]:
artists_exploded_enriched = artists_exploded.merge(data_by_artist, how = 'left', left_on = 'artists_upd',right_on = 'artists')
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]

In [33]:
artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id')['genres_upd'].apply(list).reset_index()

In [34]:
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [35]:
artists_genres_consolidated.head()

,id,genres_upd,consolidates_genre_lists
0,004G9E3EZhxxn5aE9yEQqx,"[[argentine_rock, blues_latinoamericano, latin...","[rock_nacional, blues_latinoamericano, latin_m..."
1,00AoMaoJSEEd1EZX2KnR1s,"[[adult_standards, bebop, cool_jazz, jazz, lou...","[adult_standards, swing, jazz, cool_jazz, bebo..."
2,00Bmtrh3HrbadLg1IYZTyv,[[]],[]
3,00G2UNiVKZxBxvfPaVqPMt,[[cumbia_sonidera]],[cumbia_sonidera]
4,00Ql9WL5RNqxrT9b0tjo2L,"[[adult_standards, brill_building_pop, british...","[classic_rock, british_invasion, adult_standar..."


In [36]:
spotify_df = spotify_df.merge(artists_genres_consolidated[['id','consolidates_genre_lists']], on = 'id',how = 'left')

In [37]:
spotify_df.tail()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
526570,5UbK7BJAIhkUIYYLUABLdN,The Last Supper,30,428507,0,"['""Jesus Christ Superstar"" Apostles - Original...","['1T1Hnf7jWiJPFUTi6voWwU', '479Yp6DvyXoIaCssAx...",1970-01-01,0.397,0.396,7,-8.094,1,0.0345,0.5930,0.000000,0.0997,0.232,121.013,4,"[""Jesus Christ Superstar"" Apostles - Original ...",[Jesus Christ Superstar],"['""Jesus Christ Superstar"" Apostles - Original...","""Jesus Christ Superstar"" Apostles - Original S...",NaN
526571,2itG1gLI0n9tH1DZKJbPd2,The Warrior,11,249493,0,"['""Ipi Ntombi"" 1975 Original Cast']",['2rV4kKW3Yvqh40hpDH1gpy'],1975-01-01,0.749,0.590,0,-5.699,1,0.0528,0.1580,0.003260,0.1440,0.574,116.904,4,"[""Ipi Ntombi"" 1975 Original Cast]",[Ipi Ntombi],"['""Ipi Ntombi"" 1975 Original Cast']","""Ipi Ntombi"" 1975 Original CastThe Warrior",NaN
526572,52V0laIHD3rvFZrwqDPHFe,Shosholoza,17,231733,0,"['""Ipi Ntombi"" 1975 Original Cast']",['2rV4kKW3Yvqh40hpDH1gpy'],1975-01-01,0.721,0.534,0,-10.546,1,0.0506,0.4960,0.000000,0.1110,0.923,114.075,4,"[""Ipi Ntombi"" 1975 Original Cast]",[Ipi Ntombi],"['""Ipi Ntombi"" 1975 Original Cast']","""Ipi Ntombi"" 1975 Original CastShosholoza",NaN
526573,5qhaazvxbhR8mBlo1LXTuY,Narration: Mama Tembu's Wedding,7,182733,0,"['""Ipi Ntombi"" 1975 Original Cast']",['2rV4kKW3Yvqh40hpDH1gpy'],1975-01-01,0.604,0.803,0,-6.785,1,0.4300,0.4260,0.000000,0.2480,0.916,134.034,4,"[""Ipi Ntombi"" 1975 Original Cast]",[Ipi Ntombi],"['""Ipi Ntombi"" 1975 Original Cast']","""Ipi Ntombi"" 1975 Original CastNarration: Mama...",NaN
526574,5zGtzufGU0xPwdrtellshR,Ipi Ntombi,16,217533,0,"['""Ipi Ntombi"" 1975 Original Cast']",['2rV4kKW3Yvqh40hpDH1gpy'],1975-01-01,0.694,0.802,9,-6.316,1,0.0574,0.0232,0.000006,0.1960,0.734,116.591,4,"[""Ipi Ntombi"" 1975 Original Cast]",[Ipi Ntombi],"['""Ipi Ntombi"" 1975 Original Cast']","""Ipi Ntombi"" 1975 Original CastIpi Ntombi",NaN


In [38]:
spotify_df['year'] = spotify_df['release_date'].apply(lambda x: x.split('-')[0])

In [39]:
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [40]:
ohe_cols = 'popularity'

In [41]:
spotify_df['popularity'].describe()

count    526575.000000
mean         27.193697
std          18.048252
min           0.000000
25%          13.000000
50%          27.000000
75%          40.000000
max          99.000000
Name: popularity, dtype: float64

In [42]:
# create 5 point buckets for popularity 
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [43]:
# tfidf can't handle nulls so fill any null values with an empty list
spotify_df['consolidates_genre_lists'] = spotify_df['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

In [44]:
spotify_df.sort_values('consolidates_genre_lists',ascending = False).head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,year,popularity_red
18728,1c6yom0r7k33dPWWtt30Vm,La mariposa,3,145972,0,"[""Juan D'Arienzo""]",['46beGPwcX2QEZcmuvrMqp4'],1937-04-12,0.684,0.197,6,-11.286,1,0.0807,0.966,0.0161,0.1390,0.566,128.510,4,[],[Juan D'Arienzo],[Juan D'Arienzo],[]La mariposa,"[vintage_tango, tango, orquesta_tipica]",1937,0
19639,7Eb4rSK9xAHD2xfUiIFPGC,El esquinazo,1,154462,0,"[""Juan D'Arienzo""]",['46beGPwcX2QEZcmuvrMqp4'],1937-04-12,0.827,0.369,5,-11.569,1,0.3790,0.931,0.5740,0.1720,0.955,103.157,4,[],[Juan D'Arienzo],[Juan D'Arienzo],[]El esquinazo,"[vintage_tango, tango, orquesta_tipica]",1937,0
18375,7JpEtHpiJdLvNSfxP8xc3d,Milonga del corazon (con Alberto Echague),2,149473,0,"[""Juan D'Arienzo""]",['46beGPwcX2QEZcmuvrMqp4'],1937-04-12,0.662,0.241,7,-13.571,1,0.0462,0.963,0.0569,0.0564,0.816,106.214,4,[],[Juan D'Arienzo],[Juan D'Arienzo],[]Milonga del corazon (con Alberto Echague),"[vintage_tango, tango, orquesta_tipica]",1937,0
18264,55tOwEMp0giaVqd4tFcOw8,Nada mas (con Alberto Echague),8,174393,0,"[""Juan D'Arienzo""]",['46beGPwcX2QEZcmuvrMqp4'],1937-04-12,0.774,0.198,9,-14.445,0,0.0858,0.980,0.1720,0.2080,0.813,130.897,4,[],[Juan D'Arienzo],[Juan D'Arienzo],[]Nada mas (con Alberto Echague),"[vintage_tango, tango, orquesta_tipica]",1937,1
17992,6MYfaHqxyIIngDLHAd1ADY,Pensalo bien (con Alberto Echague),5,147069,0,"[""Juan D'Arienzo""]",['46beGPwcX2QEZcmuvrMqp4'],1937-04-12,0.811,0.258,4,-12.444,0,0.1180,0.968,0.0650,0.0265,0.705,130.166,4,[],[Juan D'Arienzo],[Juan D'Arienzo],[]Pensalo bien (con Alberto Echague),"[vintage_tango, tango, orquesta_tipica]",1937,1


In [45]:
"One Hot Encoder stuff"
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [46]:
#function to build entire feature set
def create_feature_set(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final

In [47]:
#complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols).mean(axis = 0)

In [48]:
complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols)

In [49]:
complete_feature_set.head()

,genre|a_cappella,genre|acoustic_blues,genre|adult_standards,genre|album_rock,genre|alternative_country,genre|alternative_hip_hop,genre|alternative_metal,genre|alternative_pop,genre|alternative_rock,genre|american_folk_revival,genre|american_oi,genre|and,genre|appalachian_folk,genre|argentine_rock,genre|art_rock,genre|atl_hip_hop,genre|australian_hip_hop,genre|australian_rock,genre|azontobeats,genre|banda_caliente,genre|barbershop,genre|bebop,genre|big_band,genre|big_beat,genre|bluegrass_fiddle,genre|blues,genre|blues_latinoamericano,genre|blues_rock,genre|boston_rock,genre|boy_band,genre|breakcore,genre|brill_building_pop,genre|british_blues,genre|british_choir,genre|british_invasion,genre|britpop,genre|bronx_hip_hop,genre|bubblegum_pop,genre|calgary_indie,genre|calypso,genre|cambridge_choir,genre|canadian_indie,genre|canadian_singer,genre|canzone_napoletana,genre|ccm,genre|celtic,genre|chicago_blues,genre|chicago_drill,genre|chicago_rap,genre|chillwave,genre|choral,genre|christian_alternative_rock,genre|christian_music,genre|classic_french_pop,genre|classic_garage_rock,genre|classic_j,genre|classic_rock,genre|classic_soul,genre|classic_uk_pop,genre|classic_venezuelan_pop,genre|classical,genre|comic,genre|compositional_ambient,genre|contemporary_country,genre|cool_jazz,genre|country,genre|country_blues,genre|country_dawn,genre|country_pop,genre|country_road,genre|country_rock,genre|crossover_thrash,genre|crunk,genre|cumbia,genre|cumbia_sonidera,genre|dallas_indie,genre|dance_pop,genre|dance_rock,genre|deep_punk_rock,genre|delta_blues,genre|dirty_south_rap,genre|disco,genre|disco_house,genre|disco_soul,genre|disney,genre|diva_house,genre|dixieland,genre|dreamo,genre|drill,genre|drill_and_bass,genre|east_coast_hip_hop,genre|electro,genre|electro_jazz,genre|electro_swing,genre|electronica,genre|emo,genre|eugene_indie,genre|eurodance,genre|europop,genre|folk,genre|folk_rock,genre|freak_folk,genre|freakbeat,genre|french_psychedelic,genre|french_shoegaze,genre|french_synthpop,genre|funk,genre|funk_metal,genre|funk_rock,genre|g_funk,genre|gabonese_pop,genre|gangster_rap,genre|garage_rock,genre|gbvfi,genre|girl_group,genre|glam_metal,genre|grunge,genre|grupera,genre|gruperas_inmortales,genre|hard_rock,genre|hardcore,genre|hardcore_hip_hop,genre|hardcore_punk,genre|harlem_renaissance,genre|harmonica_blues,genre|hawaiian,genre|hip_hop,genre|hip_house,genre|hip_pop,genre|hollywood,genre|houston_rap,genre|hyphy,genre|idol,genre|indie_anthem,genre|indie_folk,genre|indie_garage_rock,genre|indie_pop,genre|indie_poptimism,genre|indie_rock,genre|indy_indie,genre|instrumental_funk,genre|instrumental_soul,genre|irish_country,genre|irish_folk,genre|italo_dance,genre|jangle_pop,genre|japanese_garage_rock,genre|japanese_heavy_metal,genre|japanese_psychedelic,genre|japanese_rockabilly,genre|jazz,genre|jazz_cubano,genre|jazz_venezolano,genre|jazz_violin,genre|kentucky_mountain_folk,genre|kizomba_antigas,genre|latin,genre|latin_alternative,genre|latin_christian,genre|latin_jazz,genre|latin_metal,genre|latin_pop,genre|latin_rock,genre|latin_worship,genre|lilith,genre|lounge,genre|makossa,genre|mambo,genre|medieval_folk,genre|mellow_gold,genre|melodic_hardcore,genre|memphis_blues,genre|memphis_soul,genre|merseybeat,genre|mexican_rock,genre|miami_bass,genre|miami_hip_hop,genre|minneapolis_sound,genre|modern_alternative_rock,genre|modern_hard_rock,genre|modern_rock,genre|modern_southern_rock,genre|motown,genre|movie_tunes,genre|musica_costena,genre|neo,genre|neo_mellow,genre|neoclassical_darkwave,genre|neon_pop_punk,genre|new_americana,genre|new_jack_swing,genre|new_jersey_rap,genre|new_romantic,genre|new_wave,genre|new_wave_pop,genre|nl_folk,genre|nu_jazz,genre|nu_metal,genre|nueva_ola_peruana,genre|nyhc,genre|nz_reggae,genre|old,genre|old_school_hip_hop,genre|orgcore,genre|orquesta_tipica,genre|outlaw_country,genre|paisley_underground,genre|permanent_wave,genre|peruvian_rock,genre|pet_calming,genre|philly_soul,genre|piano_rock,genre|polyphony,genre|pop

In [50]:
client_id = 'ca5d9b9710bd471baff916850b4e8490'
secret = '39e696f46e8848b292b18c242af9409f'

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [51]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [52]:
#token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=secret, redirect_uri='https://www.spotify.com/ro-en/home/')

In [53]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="ca5d9b9710bd471baff916850b4e8490", client_secret="39e696f46e8848b292b18c242af9409f"))

#gather playlist names and images. 
#images aren't going to be used until I start building a UI
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [54]:
playlist = sp.user_playlist_tracks("spotify", "7nISMCftMmrrdPgB1qxQaN")
playlist.keys() # Let's look at items and total:

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [55]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [56]:
id_name = {}

In [57]:
id_name['My Playlist #1'] = '7nISMCftMmrrdPgB1qxQaN'

In [58]:
playlist_checks = create_necessary_outputs('My Playlist #1', id_name, spotify_df)

In [59]:
playlist_checks

,artist,name,id,url,date_added
39,Mattafix,Big City Life,4stm6lf1vQBSl7Eq4Npzr0,https://i.scdn.co/image/ab67616d00001e02dfbe03...,2021-06-01 14:34:07+00:00
38,Gary Clark Jr.,Bright Lights,2NTDt9Fiqp5wTBM6cLO6Bu,https://i.scdn.co/image/ab67616d00001e025a4e89...,2021-06-01 14:33:53+00:00
37,Red Hot Chili Peppers,Give It Away,0uppYCG86ajpV2hSR3dJJ0,https://i.scdn.co/image/ab67616d00001e02153d79...,2021-06-01 14:33:36+00:00
35,Placebo,Post Blue,43Mhu1qi3qI2G2544mgelo,https://i.scdn.co/image/ab67616d00001e025a95bd...,2021-06-01 14:33:09+00:00
34,JAY-Z,No Church In The Wild,3Osd3Yf8K73aj4ySn6LrvK,https://i.scdn.co/image/ab67616d00001e024a7677...,2021-06-01 14:32:58+00:00
33,The Weeknd,Pray For Me (with Kendrick Lamar),77UjLW8j5UAGAGVGhR5oUK,https://i.scdn.co/image/ab67616d00001e02c027ad...,2021-06-01 14:32:51+00:00
32,Queen,You Don't Fool Me - Remastered 2011,1chxAv59LZcOE5FyrlUze2,https://i.scdn.co/image/ab67616d00001e0295b06b...,2021-06-01 14:32:41+00:00
30,The Knife,Marble House,59mg5dz40MDexOTFHYRBAb,https://i.scdn.co/image/ab67616d00001e023b0fe7...,2021-06-01 14:31:45+00:00
29,The Blaze,Territory,2W3EgJst61RLZtPPm3EQim,https://i.scdn.co/image/ab67616d00001e02cdc2c3...,2021-06-01 14:31:16+00:00
28,Muse,Knights of Cydonia,7ouMYWpwJ422jRcDASZB7P,https://i.scdn.co/image/ab67616d00001e0228933b...,2021-06-01 14:31:03+00:00


In [60]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [61]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlist_checks, 1.13)


In [62]:
complete_feature_set_playlist_vector

genre|a_cappella                    0.000000
genre|acoustic_blues                0.000000
genre|adult_standards               0.000000
genre|album_rock                    0.000000
genre|alternative_country           0.000000
genre|alternative_hip_hop           0.000000
genre|alternative_metal             0.000000
genre|alternative_pop               0.000000
genre|alternative_rock              0.000000
genre|american_folk_revival         0.000000
genre|american_oi                   0.000000
genre|and                           0.000000
genre|appalachian_folk              0.000000
genre|argentine_rock                0.000000
genre|art_rock                      0.000000
genre|atl_hip_hop                   0.000000
genre|australian_hip_hop            0.000000
genre|australian_rock               0.000000
genre|azontobeats                   0.000000
genre|banda_caliente                0.000000
genre|barbershop                    0.000000
genre|bebop                         0.000000
genre|big_

In [63]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [64]:
rec_top40 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)

In [65]:
rec_top40[['name','artists']]

,name,artists
488660,We Are Bulletproof Pt.2,['BTS']
153517,Ser O Parecer,"['RBD', 'Anahí', 'Dulce María', 'Maite Perroni..."
456583,Jerk It Out,['Caesars']
232518,Lolly,"['Maejor', 'Juicy J', 'Justin Bieber']"
524063,Fashion Killa,['A$AP Rocky']
121551,Ding,['Seeed']
290795,La Bilirrubina - En Vivo Estadio Olímpico De R...,['Juan Luis Guerra 4.40']
174440,Girls / Girls / Boys,['Panic! At The Disco']
430967,Six Days - Remix,"['DJ Shadow', 'Mos Def']"
198442,Oye Mi Canto,"['N.O.R.E.', 'Daddy Yankee', 'Nina Sky', 'Gems..."


In [66]:
spotify_df['consolidates_genre_lists'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                                                                                                                                                                                           524939
[classic_rock, british_invasion, adult_standards, brill_building_pop, rock-and-roll, merseybeat, rockabilly, classic_uk_pop, bubblegum_pop, folk_rock, folk, mellow_gold]                       126
[adult_standards, swing, jazz, cool_jazz, bebop, torch_song, vocal_jazz, lounge, stride]                                                                                                        100
[glam_metal, rock, hard_rock]                                                                                                                                                                    93
[classic_soul, soul, post-disco, southern_soul, urban_contemporary, motown, funk, quiet_storm, disco, philly_soul]                                                                               63
[classic_rock, hard_

In [67]:
spotify_df3 = spotify_df.join(pd.DataFrame(spotify_df.consolidates_genre_lists.tolist(),index=spotify_df.index).add_prefix('genre_'))

In [68]:
complete_feature_set['genre|rock'].value_counts()

0.000000    526185
0.247941       126
0.529283        93
0.264845        42
0.237000        38
0.426381        33
0.345220        25
0.228470        17
0.452876         6
0.417664         6
0.546785         4
Name: genre|rock, dtype: int64

In [69]:
complete_feature_set['genre|pop'].value_counts()

0.000000    526504
0.542333        27
0.431537        24
0.320825        11
0.438382         4
0.558861         3
0.280748         2
Name: genre|pop, dtype: int64

In [70]:
spotify_df3.columns

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'artists_upd_v1',
       'artists_upd_v2', 'artists_upd', 'artists_song',
       'consolidates_genre_lists', 'year', 'popularity_red', 'genre_0',
       'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6',
       'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12',
       'genre_13'],
      dtype='object')

In [71]:
#function to build entire feature set
def create_feature_set2(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    #tfidf = TfidfVectorizer()
    #tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    #genre_df = pd.DataFrame(tfidf_matrix.toarray())
    #genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    #genre_df.reset_index(drop = True, inplace=True)
    
    #df = df.join(pd.DataFrame(df.consolidates_genre_lists.tolist(),index=df.index).add_prefix('genre_'))
    #genres_df = df['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16', 'genre_17', 'genre_18', 'genre_19', 'genre_20', 'genre_21', 'genre_22']

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final


In [72]:
complete_feature_set2 = create_feature_set2(spotify_df3, float_cols=float_cols)

MemoryError: Unable to allocate 258. GiB for an array with shape (526575, 526575) and data type uint8

In [ ]:
complete_feature_set2.head()

In [ ]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set2, playlist_checks, 1.13)

In [ ]:
rec_top40 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)

In [ ]:
rec_top40[['name','artists']]

In [ ]:
spotify_df2 = spotify_df.copy()
spotify_df2['consolidates_genre_lists'] = spotify_df2['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else np.mean)

In [ ]:
spotify_df2['consolidates_genre_lists'].head()

In [ ]:
print(data_by_artist[data_by_artist['artists'] == 'Greta Van Fleet'])

In [ ]:
print(spotify_df[spotify_df['artists'] == "['Greta Van Fleet']"])

In [ ]:
#4NpFxQe2UvRCAjto3JqlSl
print(complete_feature_set[complete_feature_set['id'] == '32BEasyNQgqXQJKoVa0fUw'])
#complete_feature_set

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(spotify_df2['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
tfidf_matrix

In [ ]:
genre_df = pd.DataFrame(tfidf_matrix.toarray())

In [ ]:
genre_df.head()

In [ ]:
data_by_artist['genres'].str.contains(r'rock').sum()

In [ ]:
data_by_artist['genres'].str.contains(r'modern hard rock').sum()

In [ ]:
data_by_artist['genres'].str.contains(r'hard rock').sum()

In [ ]:
dd = data_by_artist.explode('genres')

In [ ]:
dd.head

In [ ]:
dd2 = data_by_artist.join(pd.DataFrame(data_by_artist.genres_upd.tolist(),index=data_by_artist.index).add_prefix('genre_'))

In [ ]:
dd2.tail()

In [ ]:
dd2.columns